In [1]:
import numpy as np
import xarray as xr

def parse_worksheet(text: str):
    lines = text.splitlines()
    width = max(len(s) for s in lines)
    lines = [s.ljust(width) for s in lines]
    grid = np.array([list(s) for s in lines], dtype="<U1")

    da = xr.DataArray(grid, dims=("row", "col"))

    # 1) separator columns: all spaces down the column
    sep = (da == " ").all("row").values  # shape: (col,)

    # 2) get spans of non-separator runs: [start,end)
    spans = []
    c = 0
    while c < width:
        while c < width and sep[c]:
            c += 1
        if c >= width:
            break
        start = c
        while c < width and not sep[c]:
            c += 1
        end = c
        spans.append((start, end))

    # 3) extract each problem block into a list of per-row strings
    blocks = []
    for (s, e) in spans:
        block = da.isel(col=slice(s, e))
        # join chars across col per row
        row_str = xr.apply_ufunc(
            lambda x: "".join(x).strip(),
            block,
            input_core_dims=[["col"]],
            output_core_dims=[[]],
            vectorize=True,
            dask="forbidden",
            output_dtypes=[object],
        )
        blocks.append(row_str.values.tolist())

    # 4) build xarray outputs: op and nums
    n_probs = len(blocks)
    n_rows = len(lines)

    ops = []
    nums = np.full((n_probs, n_rows - 1), np.nan, dtype=float)

    for p, rows in enumerate(blocks):
        op = rows[-1].strip()
        ops.append(op)
        for r, s in enumerate(rows[:-1]):
            if s != "":
                nums[p, r] = int(s)

    op_da = xr.DataArray(np.array(ops, dtype=object), dims=("problem",))
    nums_da = xr.DataArray(nums, dims=("problem", "row"))

    return nums_da, op_da

def grand_total(text: str):
    nums, op = parse_worksheet(text)

    sum_ans = nums.sum("row", skipna=True)
    prod_ans = nums.fillna(1).prod("row")

    ans = xr.where(op == "+", sum_ans, prod_ans)
    return ans.sum().item(), ans

# Example usage:
# total, per_problem = grand_total(puzzle_input)

In [2]:
# Read input from input.txt
with open('input.txt', 'r') as file:
  puzzle_input = file.read()

# Process the input
total, per_problem = grand_total(puzzle_input)

# Print the results
print("Grand Total:", total)
print("Per Problem Results:")
print(per_problem)

Grand Total: 5316572080628.0
Per Problem Results:
<xarray.DataArray (problem: 1000)> Size: 8kB
array([1.52470000e+04, 2.90162250e+07, 1.69000000e+03, 2.23868973e+10,
       1.17200000e+03, 1.16190480e+08, 1.96030000e+04, 1.38900000e+03,
       3.39450000e+04, 1.24002000e+05, 6.15121200e+06, 9.04000000e+02,
       9.06203750e+09, 8.10980000e+05, 2.37000000e+02, 2.35620000e+07,
       1.77870000e+07, 1.79340000e+04, 1.03000000e+02, 4.40939200e+06,
       1.06408697e+09, 1.55500000e+03, 4.96894619e+09, 1.69910000e+04,
       2.67274858e+10, 1.00720000e+04, 3.43255500e+06, 1.06300000e+03,
       9.08615788e+10, 2.39760000e+04, 2.85314400e+06, 1.32900000e+03,
       3.28315680e+07, 2.34000000e+02, 8.16000000e+02, 1.83000000e+02,
       9.32789620e+07, 1.94000000e+02, 1.28340000e+04, 7.26000000e+02,
       5.92588800e+06, 2.34951587e+10, 6.43365000e+06, 4.54860000e+05,
       1.80512640e+07, 2.25600000e+03, 2.40300000e+04, 2.80872000e+05,
       6.56640000e+04, 1.81650000e+04, 2.32700000e+03